In [6]:
# import sys
# !{sys.executable} -m pip install tensorflow==2.11.0

In [1]:
HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, Dropout, Activation, Input, Embedding, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import fasttext
import fasttext.util
import os
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import glob

2023-12-09 00:44:54.078112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 00:44:56.048358: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-09 00:44:56.048498: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-09 00:44:56.048516: W tensorfl

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
    # tf.config.set_logical_device_configuration(
    #     gpus[0],
    #     [tf.config.LogicalDeviceConfiguration(memory_limit=9216)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs;", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs; 1 Logical GPUs


2023-12-09 00:44:59.119495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 00:44:59.750521: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10785 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:8d:00.0, compute capability: 3.7


### Word Embedding

Using Wiki pretrained model

In [3]:
ft = fasttext.load_model('/home_remote/fastText/cc.en.300.bin')

In [4]:
# Function to get word embeddings for a document
def get_document_embeddings(document, max_length=100):
    # Tokenize the document and get the first max_length word vectors
    tokens = document.split()[:max_length]
    
    # Apply zero-padding if the document is shorter than max_length
    if len(tokens) < max_length:
        padding_count = max_length - len(tokens)
        tokens.extend(['<PAD>'] * padding_count)

    # Get word embeddings for each token
    embeddings = [ft.get_word_vector(token) for token in tokens]

    return np.array(embeddings)


# Convolutional Neural Network

# Preprocess train & test data

In [10]:
positives_df_path = os.path.join(HOME_DIR, "positive_df.pkl")
negatives_df_path = os.path.join(HOME_DIR, "negative_df.pkl")

positives = pd.read_pickle(positives_df_path)
negatives = pd.read_pickle(negatives_df_path)

positives['Label'] = 1
negatives['Label'] = 0

#concatenate the two dataframes
df = pd.concat([positives, negatives], ignore_index=True)

In [11]:
subject = df.TrainSubjectId.unique()
#split the subjects into train and test
train_subject, test_subject = train_test_split(subject, test_size=0.2, random_state=42, shuffle=True)
df_train = df[df.TrainSubjectId.isin(train_subject)]
df_test = df[df.TrainSubjectId.isin(test_subject)]
df_train = df_train.sample(frac=1, random_state=42)
df_test = df_test.sample(frac=1, random_state=42)
y_train = df_train.Label
y_test = df_test.Label
X_train = df_train['Text']
X_test = df_test['Text']

In [ ]:
#convert X_train to word embeddings using get_document_embeddings function
X_train = np.array([get_document_embeddings(document) for document in X_train])
X_test = np.array([get_document_embeddings(document) for document in X_test])
X_train.shape, X_test.shape

((237427, 100, 300), (59608, 100, 300))

In [15]:
#convert y_train and y_test to numpy array with shape (n, 1)
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)
y_train.shape, y_test.shape

((237427, 1), (59608, 1))

## Save train and test data to disk

In [36]:
def save_data_to_disk(np_array, file_name_prefix, file_type="npy"):
    """
    Split the np_array into 10 chunks and save them to csv files
    """
    chunks = np.array_split(np_array, 10)
    for i in range(len(chunks)):
        filename = os.path.join(HOME_DIR, "master_thesis/data", "{}_{}.{}".format(file_name_prefix, i + 1, file_type))
        if file_type == "npy":
            np.save(filename, chunks[i])
        elif file_type == "csv":
            np.savetxt(filename, chunks[i])
        else:
            raise ValueError("file_type must be either npy or csv")

In [ ]:
# X_train_2D = X_train.reshape(X_train.shape[0], -1)
# save_data_to_disk(X_train_2D, "X_train_2D")
# X_test_2D = X_test.reshape(X_test.shape[0], -1)
# save_data_to_disk(X_test_2D, "X_test_2D")

# save_data_to_disk(y_train, "y_train")
# save_data_to_disk(y_test, "y_test")

In [37]:
save_data_to_disk(X_train, "X_train", file_type="npy")

In [38]:
save_data_to_disk(y_train, "y_train", file_type="npy")

# Train model

## Create `tf.data.Dataset` object to store train data

In [28]:
# def read_npy_file(X_path, y_path):
#     X = np.load(X_path)
#     y = np.load(y_path)
#     return X, y, X.shape, y.shape

# def load_and_parse_data_from_npy_file(X_path, y_path):
#     # "temp[0]" is X, "temp[1]" is y, "temp[2]" is X.shape, "temp[3]" is y.shape
#     temp = tf.py_function(read_npy_file, inp=[X_path, y_path], Tout=(tf.float32, tf.int32, tf.int32, tf.int32))
#     X_tensor = tf.reshape(temp[0], temp[2])
#     y_tensor = tf.reshape(temp[1], temp[3])
#     return X_tensor, y_tensor

In [33]:
# X_file_pattern = os.path.join(HOME_DIR, "master_thesis/data/X_*.npy")
# X_file_list = glob.glob(X_file_pattern)
# X_file_list.sort()

# y_file_pattern = os.path.join(HOME_DIR, "master_thesis/data/y_*.npy")
# y_file_list = glob.glob(y_file_pattern)
# y_file_list.sort()

# train_dataset_file_paths = tf.data.Dataset.from_tensor_slices((X_file_list, y_file_list))
# train_dataset = train_dataset_file_paths.map(load_and_parse_data_from_npy_file)
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

In [43]:
def generator_function(X_train_path, y_train_path):
    X_train_file_list = glob.glob(X_train_path)
    X_train_file_list.sort()
    y_train_file_list = glob.glob(y_train_path)
    y_train_file_list.sort()

    for npy_file_x, npy_file_y in zip(X_train_file_list, y_train_file_list):
        data_x = np.load(npy_file_x, mmap_mode='r')
        data_y = np.load(npy_file_y, mmap_mode='r')

        batch_size = 32
        for i in range(0, len(data_x), batch_size):
            yield data_x[i:i + batch_size], data_y[i:i + batch_size]

X_train_filename_pattern = os.path.join(HOME_DIR, "master_thesis/data/X_train*.npy")
X_train_shape = (None, 100, 300)
y_train_filename_pattern = os.path.join(HOME_DIR, "master_thesis/data/y_train*.npy")
y_train_shape = (None, 1)
dataset = tf.data.Dataset.from_generator(
    generator_function, 
    args=(X_train_filename_pattern, y_train_filename_pattern), 
    output_signature=(
        tf.TensorSpec(shape=X_train_shape, dtype=tf.float32),
        tf.TensorSpec(shape=y_train_shape, dtype=tf.int32)
    )
)

## Construct Sequential models

In [44]:
# Custom CReLU activation function
def crelu(x):
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=-1)

In [45]:
# Word embedding dimensionality
word_vector_dimensions = 300

# Model
model = Sequential()

# Convolutional Layer
model.add(Conv1D(filters=100, kernel_size=2, input_shape=(100, 300)))
model.add(Activation(crelu))
model.add(Activation(lambda x: x * -1))  # Negated activation for CReLU

# 1-Max Pooling Layer
model.add(MaxPooling1D(pool_size=99))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(200))

# Dropout Layer
model.add(Dropout(0.4))

# 3 Fully Connected Layers
model.add(Dense(100, activation=crelu))
model.add(Dense(50, activation=crelu))
model.add(Dense(25, activation=crelu))

# Final Layer
model.add(Dense(2, activation='softmax'))
model.add(Dense(1, activation='softmax'))

# Display the model summary
model.summary()

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 99, 100)           60100     
                                                                 
 activation_6 (Activation)   (None, 99, 200)           0         
                                                                 
 activation_7 (Activation)   (None, 99, 200)           0         
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 1, 200)           0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 200)               0         
                                                                 
 dense_18 (Dense)            (None, 200)               40200     
                                                      

## Train model with `tf.data.DataSet`

In [ ]:
model.fit(dataset, epochs=30)

In [10]:
model

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(64)

In [13]:
txt = "i'm depressed and want to kill myself."
vc = get_document_embeddings(txt)
vc.reshape(1, 100, 300)

array([[[-0.00728162, -0.05424538,  0.03505474, ...,  0.00139213,
         -0.1551645 , -0.08451691],
        [ 0.00735422, -0.0123115 ,  0.00735966, ...,  0.07680896,
         -0.00778688,  0.03561359],
        [ 0.00823911, -0.08990277,  0.02652529, ..., -0.01159137,
         -0.04112864,  0.03625222],
        ...,
        [-0.00627562, -0.02137895,  0.01162977, ..., -0.02386858,
          0.02757748, -0.00151885],
        [-0.00627562, -0.02137895,  0.01162977, ..., -0.02386858,
          0.02757748, -0.00151885],
        [-0.00627562, -0.02137895,  0.01162977, ..., -0.02386858,
          0.02757748, -0.00151885]]], dtype=float32)

In [14]:
model.predict(vc.reshape(1, 100, 300))

1/1 [==============================] - 0s 33ms/step


array([[1.]], dtype=float32)